# VAE Training

In [1]:
import torch as t
import torch.nn.functional as F
from tqdm import tqdm
from models.VariationalAutoencoder import VariationalAutoencoder
from torchvision import datasets, transforms

bs = 32
train_ds = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
train_dl = t.utils.data.DataLoader(dataset=train_ds, batch_size=bs, shuffle=True, drop_last=True)

device = t.device('cuda' if t.cuda.is_available() else 'cpu')
model = VariationalAutoencoder(train_ds[0][0][None], in_c=1, enc_out_c=[32, 64, 64, 64],
                    enc_ks=[3, 3, 3, 3], enc_pads=[1, 1, 0, 1], enc_strides=[1, 2, 2, 1],
                    dec_out_c=[64, 64, 32, 1], dec_ks=[3, 3, 3, 3], dec_strides=[1, 2, 2, 1],
                    dec_pads=[1, 0, 1, 1], dec_op_pads=[0, 1, 1, 0], z_dim=2)
model.cuda(device)
model.train()

VariationalAutoencoder(
  (enc_conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(nega

In [2]:
def vae_kl_loss(mu, log_var):
    return -.5 * t.sum(1 + log_var - mu ** 2 - log_var.exp())

def vae_loss(y_pred, mu, log_var, y_true, r_loss_factor=100):
    r_loss = F.binary_cross_entropy(y_pred, y_true, reduction='sum')
    kl_loss = vae_kl_loss(mu, log_var)
    return r_loss_factor * r_loss + kl_loss

In [3]:
lr = .0005
for epoch in tqdm(range(20)):
    optimizer = t.optim.Adam(model.parameters(), lr=lr / (epoch * 2 + 1), betas=(.9, .99), weight_decay=1e-2)
    for i, (data, _) in enumerate(train_dl):
        data = data.to(device)
        optimizer.zero_grad()
        pred, mu, log_var = model(data)
        loss = vae_loss(pred, mu, log_var, data)
        loss.backward()
        optimizer.step()
        if i % 33 == 0:
            print(loss)

print(loss)

  0%|          | 0/20 [00:00<?, ?it/s]

tensor(3661121.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1680244.6250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1436101., device='cuda:0', grad_fn=<AddBackward0>)
tensor(1181363.1250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1196160.3750, device='cuda:0', grad_fn=<AddBackward0>)
tensor(848027.3750, device='cuda:0', grad_fn=<AddBackward0>)
tensor(817010.9375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(854536.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(708343.3125, device='cuda:0', grad_fn=<AddBackward0>)
tensor(667996.8750, device='cuda:0', grad_fn=<AddBackward0>)
tensor(661654.3125, device='cuda:0', grad_fn=<AddBackward0>)
tensor(579668.1250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(640547.4375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(610358.9375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(657156.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(587984.0625, device='cuda:0', grad_fn=<AddBackward0>)
tensor(599292.1250, dev

  5%|▌         | 1/20 [00:43<13:47, 43.54s/it]

tensor(503492.0625, device='cuda:0', grad_fn=<AddBackward0>)
tensor(548645.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(599311.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(558159.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(559970.0625, device='cuda:0', grad_fn=<AddBackward0>)
tensor(516686.4688, device='cuda:0', grad_fn=<AddBackward0>)
tensor(492067.5938, device='cuda:0', grad_fn=<AddBackward0>)
tensor(495852.2188, device='cuda:0', grad_fn=<AddBackward0>)
tensor(544231.0625, device='cuda:0', grad_fn=<AddBackward0>)
tensor(540126.5625, device='cuda:0', grad_fn=<AddBackward0>)
tensor(494645.0938, device='cuda:0', grad_fn=<AddBackward0>)
tensor(530919.9375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(502906.4375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(583522.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(516078.3125, device='cuda:0', grad_fn=<AddBackward0>)
tensor(501771.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(548026.2500, devi

 10%|█         | 2/20 [01:26<12:59, 43.31s/it]

...

tensor(493854.0938, device='cuda:0', grad_fn=<AddBackward0>)
tensor(468791.2812, device='cuda:0', grad_fn=<AddBackward0>)
tensor(486138.1562, device='cuda:0', grad_fn=<AddBackward0>)
tensor(446362.6250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(498076.8438, device='cuda:0', grad_fn=<AddBackward0>)
tensor(496157.8438, device='cuda:0', grad_fn=<AddBackward0>)
tensor(475429.4375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(508219.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(537365.1250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(514909., device='cuda:0', grad_fn=<AddBackward0>)
tensor(454199.8438, device='cuda:0', grad_fn=<AddBackward0>)
tensor(502949.5625, device='cuda:0', grad_fn=<AddBackward0>)
tensor(483019.3125, device='cuda:0', grad_fn=<AddBackward0>)
tensor(493767.1562, device='cuda:0', grad_fn=<AddBackward0>)
tensor(459879.8438, device='cuda:0', grad_fn=<AddBackward0>)
tensor(447755.6562, device='cuda:0', grad_fn=<AddBackward0>)
tensor(496561.2188, device='

 95%|█████████▌| 19/20 [13:30<00:42, 42.26s/it]

tensor(435721.0625, device='cuda:0', grad_fn=<AddBackward0>)
tensor(502921.8438, device='cuda:0', grad_fn=<AddBackward0>)
tensor(537985.1250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(471795.4062, device='cuda:0', grad_fn=<AddBackward0>)
tensor(502709.4375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(496579.1562, device='cuda:0', grad_fn=<AddBackward0>)
tensor(495587.2812, device='cuda:0', grad_fn=<AddBackward0>)
tensor(499000.5625, device='cuda:0', grad_fn=<AddBackward0>)
tensor(484532.3750, device='cuda:0', grad_fn=<AddBackward0>)
tensor(448906.8438, device='cuda:0', grad_fn=<AddBackward0>)
tensor(498294.7812, device='cuda:0', grad_fn=<AddBackward0>)
tensor(463099.3125, device='cuda:0', grad_fn=<AddBackward0>)
tensor(483027.2812, device='cuda:0', grad_fn=<AddBackward0>)
tensor(446956.6250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(478200.8125, device='cuda:0', grad_fn=<AddBackward0>)
tensor(515652.9062, device='cuda:0', grad_fn=<AddBackward0>)
tensor(494237.9688, devi

100%|██████████| 20/20 [14:12<00:00, 42.20s/it]

tensor(507324.0312, device='cuda:0', grad_fn=<AddBackward0>)


In [4]:
t.save(model.state_dict(), 'models/state_dicts/03_02.pth')